In [34]:
import re
import json
import pickle

from unittest import result
from itertools import combinations
import matplotlib.pyplot as plt
import math
import pickle

In [35]:
%run text_normalizer.ipynb
%run news.ipynb

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kiana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kiana\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
def creating_dictionary_and_vectors():
 
  space_codepoints ='\u0020\u2000-\u200F\u2028-\u202F'
  persian_alpha_codepoints = '\u0621-\u0628\u062A-\u063A\u0641-\u0642\u0644-\u0648\u064E-\u0651\u0655\u067E\u0686\u0698\u06A9\u06AF\u06BE\u06CC'
  persian_num_codepoints = '\u06F0-\u06F9'
  punctuation_marks_codepoints = '\u060C\u061B\u061F\u0640\u066A\u066B\u066C'
  additional_arabic_characters_codepoints = '\u0629\u0643\u0649-\u064B\u064D\u06D5'
  arabic_numbers_codepoints = '\u0660-\u0669'
    
  journals_data = read_JSON('data/journals.json')
  all_news = []
  all_journals = {}
  doc_ID = 1

  for journal in journals_data["articles"]:
        keywords_as_string = ''.join(journal["articleKeywords"])
        result=re.search('^[\s,'+persian_alpha_codepoints+additional_arabic_characters_codepoints
                         +arabic_numbers_codepoints+persian_num_codepoints+']*$', journal["articleTitle"]+keywords_as_string)

        if not result:
             # print(result)
            all_journals[doc_ID] = journal
            doc_ID += 1

  
 
  all_docs_count = len(all_journals)
  
  print(all_docs_count)


  words_dictionary = {} # term - document frequency
  term_postings = {} # term - postings list
  # previous_doc_ID = -1

  for journal_id in all_journals:
        journal = all_journals[journal_id]
        doc_object = NewsDocument(journal_id,journal["articleTitle"],journal["articleKeywords"],all_docs_count,journal["journalLink"],1)
        all_news.append(doc_object)

        tokens = doc_object.get_preproccessed_tokens()
       
        for word in tokens:
          if word in words_dictionary:


            term_postings[word].add(doc_object)
            words_dictionary[word] = len(term_postings[word])

          else:

           term_postings[word] = set()
           term_postings[word].add(doc_object)
           words_dictionary[word] = len(term_postings[word])
          #  previous_doc_ID = doc_ID

    # print(words_dictionary)
  i = 0
  for news in all_news:
    print(i)
    news.create_vector(words_dictionary)
    i+= 1


  #saving data

  file = open("words_dictionary.pkl","wb")
  pickle.dump(words_dictionary, file) 
  file.close()
  file = open("all_news.pkl","wb")
  pickle.dump(all_news, file) 
  file.close()
  file = open("term_postings.pkl","wb")
  pickle.dump(term_postings, file) 
  file.close()

  return words_dictionary, all_news, term_postings

In [37]:
def read_JSON(path):
    with open(path, encoding='utf-8') as fh:
        articles_data = json.load(fh)
    return articles_data

In [38]:
def get_data_from_files():

  #oppening data files 
  print("getting dictionary ......")
  file = open("pickleFiles/words_dictionary.pkl", "rb")
  words_dictionary = pickle.load(file)
  file.close()
  print("getting documents ......")
  file = open("pickleFiles/all_news.pkl", "rb")
  all_news = pickle.load(file)
  file.close()
#   print("getting champion list ......\n")
#   file = open("pickleFiles/champion_list.pkl", "rb")
#   term_champion_list = pickle.load(file)
#   file.close()
  print("getting term postings ......\n")
  file = open("pickleFiles/term_postings.pkl", "rb")
  term_postings = pickle.load(file)
  file.close()
  print("Done \n")

  return words_dictionary, all_news, term_postings

In [39]:
def suggest_news_to_query(results_dict, suggestion_string):
    
    

    fp = open(suggestion_string+".txt","w",encoding='utf-8',newline='')
    # count = 1
    for item in results_dict:
        
        fp.write("\n\n******************  "+ item +" *******************\n\n")
        for res in results_dict[item]:
            fp.write(res+"\n")
        
    fp.close()
        # count += 1

In [40]:
def search_query(journal_object, count, dictionary, all_news, term_postings):
  tokens = journal_object.get_preproccessed_tokens()
  journal_object.create_vector(dictionary)
  journal_object.find_cosine_distances_from_all_news(all_news, term_postings)
  # count = input("Enter number of k for showing top k results\n")
  top_news,cosines,urls = journal_object.get_top_nearest_news(count=int(count))
  # print(cosines)
  return top_news,cosines,urls

In [41]:
def get_preprocessed_articles(all_docs_count):
  articles_data = read_JSON('data/scholars.json')  
  all_articles = []
  doc_ID = -1
  for author in articles_data:
    for paper in author["articles"]:
#         print(paper)
        doc_object = None
        if "url" in paper.keys():
            doc_object = NewsDocument(doc_ID, paper["title"],paper["keywords"],all_docs_count,paper["url"],0)
        else:
            doc_object = NewsDocument(doc_ID, paper["title"],paper["keywords"],all_docs_count,"NO URL",0)
        all_articles.append(doc_object)
        doc_ID -=1
  return all_articles
            

In [42]:
if __name__ == "__main__":
    
    

#   words_dictionary, all_news, term_postings = get_data_from_files()
  # words_dictionary, all_news = get_data_from_files()
  words_dictionary, all_news, term_postings = creating_dictionary_and_vectors()
  print(words_dictionary)
  print(len(words_dictionary))


2949
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
2

1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071


In [43]:
    all_articles = get_preprocessed_articles(len(all_news))
    count = input("Enter number of k for showing top k results\n")
    result = {}
    for article in all_articles:
      result[article.get_title()]= []
      top_news,cosines,urls = search_query(article, count, words_dictionary, all_news, term_postings)
      i=0
      for news in top_news:
        res = "ID: {}[\ntitle: {}\ncosine: {}\nurl: {}\n]\n".format(news.get_doc_ID(),news.get_title(), cosines[i], urls[i])
#         print(res)
        result[article.get_title()].append(res)
        i+=1
    suggest_news_to_query(result, "suggest_journals_to_articles")

Enter number of k for showing top k results
10
